In [6]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
import pickle

In [ ]:
data_directory = '/content/drive/MyDrive/Colab Notebooks/Autoencoders/stl10_binary/'
train_images_file = np.fromfile(os.path.join(data_directory, 'train_X.bin'), dtype=np.uint8)
train_labels_file = np.fromfile(os.path.join(data_directory, 'train_y.bin'), dtype=np.uint8)
test_images_file = np.fromfile(os.path.join(data_directory, 'test_X.bin'), dtype=np.uint8)
test_labels_file = np.fromfile(os.path.join(data_directory, 'test_y.bin'), dtype=np.uint8)

# Reshape and normalize the training images

train_images = train_images_file.reshape((-1, 3, 96, 96)).transpose((0, 3, 2, 1))
train_images = train_images.astype(np.float32) / 255.0

In [ ]:
# Split the dataset into training and validation sets

train_images, validation_images, train_labels, validation_labels = train_test_split(
train_images, train_labels_file, test_size=0.2, random_state=42
)
# Use pretrained autoencoder's encoder as a feature extractor
autoencoder_best = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/DL_Ass2_M22AI529/model.sav', 'rb'))
pretrained_autoencoder = autoencoder_best
encoder = pretrained_autoencoder.layers[1] # Assuming the encoder is the second layer

# Extract features from images

train_features = pretrained_autoencoder.predict(train_images)
validation_features = pretrained_autoencoder.predict(validation_images)
test_features = pretrained_autoencoder.predict(test_images_file)

In [ ]:
# Build MLP classifiers

hidden_layers_3 = [256, 128, 64]
hidden_layers_5 = [512, 256, 128, 64, 32]

def build_mlp_classifier(hidden_layers):
  model = models.Sequential()
  model.add(layers.Flatten(input_shape=train_features.shape[1:]))
  for units in hidden_layers:
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  return model

# Build MLP classifier with 3 hidden layers

mlp_classifier_3 = build_mlp_classifier(hidden_layers_3)
mlp_classifier_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_classifier_3.fit(train_features, train_labels, validation_data=(validation_features, validation_labels), epochs=10)

# Build MLP classifier with 5 hidden layers

mlp_classifier_5 = build_mlp_classifier(hidden_layers_5)
mlp_classifier_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_classifier_5.fit(train_features, train_labels, validation_data=(validation_features, validation_labels), epochs=10)

In [ ]:
# Fine-tune classifiers with different training sample percentages

train_samples = [0.01, 0.1, 0.2, 0.4, 0.6]
classifiers = [mlp_classifier_3, mlp_classifier_5]
results = {}

for classifier in classifiers:
  classifier_results = []
  for sample_percent in train_samples:
    # Randomly sample a subset of training data
    sample_indices = np.random.choice(len(train_features), int(sample_percent * len(train_features)), replace=False)
    sampled_features = train_features[sample_indices]
    sampled_labels = train_labels[sample_indices]
    # Fine-tune classifier on sampled data
    classifier.fit(sampled_features, sampled_labels, epochs=5, verbose=0)

    # Evaluate classifier on test set
    test_loss, test_accuracy = classifier.evaluate(test_features, test_labels_file, verbose=0)

    classifier_results.append(test_accuracy)

    results[classifier.name] = classifier_results

In [ ]:
# Evaluate performance using confusion matrix and AUC-ROC curve

def evaluate_performance(classifier, features, labels):
  predictions = classifier.predict(features)
  y_pred = np.argmax(predictions, axis=1)
  confusion_mtx = confusion_matrix(labels, y_pred)
  auc_roc = roc_auc_score(labels, predictions, multi_class='ovr')
  return confusion_mtx, auc_roc

# Evaluate performance for MLP classifiers

confusion_mtx_3, auc_roc_3 = evaluate_performance(mlp_classifier_3, test_features, test_labels_file)
confusion_mtx_5, auc_roc_5 = evaluate_performance(mlp_classifier_5, test_features, test_labels_file)

# Evaluate performance for fine-tuned classifiers

confusion_mtx_results = {}
auc_roc_results = {}

for classifier_name, classifier_results in results.items():
  classifier = classifiers[classifier_name]
  confusion_mtx, auc_roc = evaluate_performance(classifier, test_features, test_labels_file)
  confusion_mtx_results[classifier_name] = confusion_mtx
  auc_roc_results[classifier_name] = auc_roc

In [ ]:
# Implement a different architecture for improvement (e.g., CNN)

def build_cnn_classifier():
  base_model = VGG16(include_top=False, weights='imagenet', input_shape=train_images.shape[1:])
  for layer in base_model.layers:
  layer.trainable = False
  model = models.Sequential()
  model.add(base_model)
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  return model

cnn_classifier = build_cnn_classifier()
cnn_classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_classifier.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=10)

# Evaluate CNN classifier performance

cnn_confusion_mtx, cnn_auc_roc = evaluate_performance(cnn_classifier, test_images_file, test_labels_file)

In [ ]:
import os
import tarfile

# Define the paths and filenames

tar_file_path = 'stl10_binary.tar.gz'
extracted_folder = ''

# Create the destination folder if it doesn't exist

os.makedirs(extracted_folder, exist_ok=True)

# Extract the tar.gz file

with tarfile.open(tar_file_path, 'r:gz') as tar:
  tar.extractall(extracted_folder)

print('Extraction completed.')